# Demo

## 如何使用fire 工具

In [ ]:
import fire
from .prompts import fix
import os
from zxftools.llms import get_llm

def ai_fix(file_path=''):
    assert os.path.isfile(file_path)
    llm = get_llm(model=os.environ.get('aigen_llm') or 'gpt-3.5-turbo-0613')

    with open(file_path,'r') as f:
        code = f.read()
    result = llm.complete(fix.format(code=code))

    with open(file_path,'w') as f:
        f.write(result.text)

if __name__ == '__main__':
    fire.Fire(ai_fix)



## 编写一个装饰器

In [ ]:
import functools

def decorator(a = None):
    """
    一个装饰器的编写demo
    """
    def outer_packing(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            result = func(*args, **kwargs)
            print(a,'a')
            print(func.__name__)  # 函数名
            print(args)  # (1, 2)
            print(kwargs)  # {'c': 3}
            return result
        return wrapper
    return outer_packing

hard_dependencies = ('time',)
check_package(hard_dependencies=hard_dependencies,check=False)



In [ ]:
from openai import OpenAI
import os
client = OpenAI(base_url="https://api.bianxieai.com/v1",
                api_key=os.environ.get('bianxieai_API_KEY'))

completion = client.chat.completions.create(
  model="claude-3-5-sonnet-20240620",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)
if __name__ == '__main__':
    print(completion.choices[0].message)

In [ ]:
import os
from typing import Optional, List, Mapping, Any

from llama_index.core.llms.callbacks import llm_completion_callback
from llama_index.core.llms import (
    CustomLLM,
    CompletionResponse,
    CompletionResponseGen,
    LLMMetadata,
)

from openai import OpenAI

class BianXieLLM(CustomLLM):
    context_window: int = 3900
    num_output: int = 256
    model: str = ''
    temperature: int = 0.2
    top_p: float = 0.9
    top_k: int = 50
    penalty_score: float = 1.0
    stop = []

    def __init__(self, temperature=0.2, model='claude-3-5-sonnet-20240620',api_key=None):
        super().__init__()
        object.__setattr__(self, 'custom_openai_client',OpenAI(base_url="https://api.bianxieai.com/v1",
                api_key=api_key or os.environ.get('bianxieai_API_KEY'))
 )
        self.model = model
        self.temperature = temperature

    @property
    def metadata(self) -> LLMMetadata:
        return LLMMetadata(
            context_window=self.context_window,
            num_output=self.num_output,
            model_name=self.model,
        )

    @llm_completion_callback()
    def complete(self, prompt: str, **kwargs: Any) -> CompletionResponse:
        completion = self.custom_openai_client.chat.completions.create(
              model=self.model,
              temperature = self.temperature,
              messages=[{"role": "user", "content": prompt}],
              **kwargs
            )
        return CompletionResponse(text=completion.choices[0].message.content)

    @llm_completion_callback()
    def stream_complete(self, prompt: str, **kwargs: Any) -> CompletionResponseGen:
        completion = self.custom_openai_client.chat.completions.create(
              model=self.model,
              temperature = self.temperature,
              messages=[{"role": "user", "content": prompt}],
              **kwargs
            )
        #return CompletionResponse(text=completion.choices[0].message.content)
        response_generate = [1,2,3]
        response = ""
        for token in response_generate:
            response += token
            yield CompletionResponse(text=response, delta=token)


In [ ]:
    @staticmethod
    def thread_list(f, params: types.GeneratorType or list, processes=4, chunksize: int = 1):
        """
        1 必须到 if __name__ == '__main__': 下执行
        """
        pool = Pool(processes)
        if isinstance(params, types.GeneratorType):
            results = pool.imap(f, params, chunksize=chunksize)
        else:
            results = pool.map(f, params, chunksize=chunksize)
        pool.close()
        return results